<a href="https://colab.research.google.com/github/bagavijit418/POC_Startified_CV/blob/master/AG_case_Study_Stratified_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study of the Titanic Data Set: A Classification Problem Statement using Stratified Cross-Validation

Dataset

Test: https://s3.amazonaws.com/acadgildsite/wordpress_images/datasets/titanic/test.csv

Train:

https://s3.amazonaws.com/acadgildsite/wordpress_images/datasets/titanic/train.csv

## Read DataSet

In [0]:
import pandas as pd

train_ds=pd.read_csv("https://s3.amazonaws.com/acadgildsite/wordpress_images/datasets/titanic/train.csv")

In [0]:
test_ds=pd.read_csv("https://s3.amazonaws.com/acadgildsite/wordpress_images/datasets/titanic/test.csv")

In [0]:
train_ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
test_ds.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
print(train_ds.shape)
print(test_ds.shape)

(891, 12)
(418, 11)


In [0]:
train_ds.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [0]:
test_ds.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

as per the above dataset, test data does not have 'survived' column which is our traget column. We have to train the  model using training data and then predict if a passenger will survive or not best on the test data

In [0]:
# Creating X and y 
y=train_ds['Survived']
X=train_ds.drop('Survived',axis=1)

## Data Preprocessing

### Check for missing values and imputing value in missing fields

In [0]:
#Checking for missing values 

X.isna().any()


PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

#### Embarked Column

In [0]:
# Checking distinct values under 'Embarked' column

X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [0]:
#Checking how many rows  have na values
X['Embarked'].isna().sum()

2

In [0]:
#Checking what are those rows
X[X['Embarked'].isna()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [0]:
#train dataset
X['Embarked'].fillna('S',inplace=True)

#test dateset
test_ds['Embarked'].fillna('S',inplace=True)

In [0]:
X['Embarked'].isna().any()

test_ds['Embarked'].isna().any()

False

#### Age Column

In [0]:
X['Age'].fillna(X['Age'].median(),inplace=True)

In [0]:
test_ds['Age'].fillna(test_ds['Age'].median(),inplace=True)

#### Cabin column

In [0]:
import numpy as np
#we will fill up the value of cabin with 0 if it has null value, else , we have to fill with 1

X.loc[:,['Cabin']]=np.where(X['Cabin'].isna(),0,1)


In [0]:
X['Cabin'].isna().any()

False

In [0]:
X['Cabin'].value_counts()

0    687
1    204
Name: Cabin, dtype: int64

In [0]:
# Test Dataset

test_ds.loc[:,['Cabin']]=np.where(test_ds['Cabin'].isna(),0,1)

In [0]:
test_ds['Cabin'].value_counts()

0    327
1     91
Name: Cabin, dtype: int64

 All 'NaN' values have been replaced with 0 and non null values with 1

####Fare column (for test dataset)

In [0]:
#training ds does not have a null value in fare column , but the same is present under test dataset

test_ds['Fare'].fillna(test_ds['Fare'].median(),inplace=True)

### Dummy Treatment

Perform dummy treatment on ‘Pclass’, ‘Sex’, ‘SibSp’, ‘Parch’, ‘Embarked’ for both test and train sets and drop the original columns

In [0]:
''' 
from sklearn.preprocessing import OneHotEncoder,Binarizer,CategoricalEncoder,LabelEncoder
sample =X[['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']]

ohe=OneHotEncoder()

p_class_one_hot= ohe.fit_transform(sample)

'''

" \nfrom sklearn.preprocessing import OneHotEncoder,Binarizer,CategoricalEncoder,LabelEncoder\nsample =X[['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']]\n\nohe=OneHotEncoder()\n\np_class_one_hot= ohe.fit_transform(sample)\n\n"

In [0]:
#train
p_class_dum=pd.get_dummies(X['Pclass'],prefix='pclass')

#p_class_dum.head()
#test

test_p_class_dum=pd.get_dummies(test_ds['Pclass'],prefix='pclass')

In [0]:
sex_dum=pd.get_dummies(X['Sex'],prefix='sex')

test_sex_dum=pd.get_dummies(test_ds['Sex'],prefix='sex')
#sex_dum.head()

In [0]:
parch_dum=pd.get_dummies(X['Parch'],prefix='purch')

test_parch_dum=pd.get_dummies(test_ds['Parch'],prefix='purch')
#parch_dum.head()

In [0]:
sibsp_dum=pd.get_dummies(X['SibSp'],prefix='sibsp')

test_sibsp_dum=pd.get_dummies(test_ds['SibSp'],prefix='sibsp')
#sibsp_dum.head()

In [0]:
embarked_dum=pd.get_dummies(X['Embarked'],prefix='embark')

test_embarked_dum=pd.get_dummies(test_ds['Embarked'],prefix='embark')
#embarked_dum.head()

In [0]:
#concat dummy columns to original dataset and drop original columns
X=pd.concat([X,p_class_dum,sex_dum,parch_dum,sibsp_dum,embarked_dum],axis=1)

X.drop(['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked'],axis=1,inplace=True)

In [0]:
X.columns

Index(['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'pclass_1',
       'pclass_2', 'pclass_3', 'sex_female', 'sex_male', 'purch_0', 'purch_1',
       'purch_2', 'purch_3', 'purch_4', 'purch_5', 'purch_6', 'sibsp_0',
       'sibsp_1', 'sibsp_2', 'sibsp_3', 'sibsp_4', 'sibsp_5', 'sibsp_8',
       'embark_C', 'embark_Q', 'embark_S'],
      dtype='object')

In [0]:
#concat dummy columns to original dataset and drop original columns for test dataset
test_ds=pd.concat([test_ds,test_p_class_dum,test_sex_dum,test_parch_dum,test_sibsp_dum,test_embarked_dum],axis=1)

In [0]:
test_ds.drop(['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked'],axis=1,inplace=True)

In [0]:
test_ds.columns

Index(['PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'pclass_1',
       'pclass_2', 'pclass_3', 'sex_female', 'sex_male', 'purch_0', 'purch_1',
       'purch_2', 'purch_3', 'purch_4', 'purch_5', 'purch_6', 'purch_9',
       'sibsp_0', 'sibsp_1', 'sibsp_2', 'sibsp_3', 'sibsp_4', 'sibsp_5',
       'sibsp_8', 'embark_C', 'embark_Q', 'embark_S'],
      dtype='object')

In [0]:
# drop few unnecessary columns from both training and test ds

X.drop(['PassengerId', 'Name','Ticket'],axis=1,inplace=True)


In [0]:
X.shape

(891, 25)

In [0]:
test_ds.drop(['PassengerId', 'Name','Ticket','purch_9',],axis=1,inplace=True) #parch_9 additionally dropped from test coulm because it was not present on training dataset

In [0]:
test_ds.shape

(418, 25)

### Perform bining for continuous features 

#### Age Column

In [0]:
# Performing median binning of age column and delete original column

#bin for age =4
X['age_cat']=pd.qcut(X['Age'],q=4,labels=False)

In [0]:
X['fare_cat']=pd.qcut(X['Fare'],q=4,labels=False)

In [0]:
test_ds['age_cat']=pd.qcut(test_ds['Age'],q=4,labels=False)
test_ds['fare_cat']=pd.qcut(test_ds['Fare'],q=4,labels=False)

In [0]:
X.drop(['Age','Fare'],axis=1,inplace=True)

In [0]:
test_ds.drop(['Age','Fare'],axis=1,inplace=True)

In [0]:
X.head().T

,0,1,2,3,4
Cabin,0,1,0,1,0
pclass_1,0,1,0,1,0
pclass_2,0,0,0,0,0
pclass_3,1,0,1,0,1
sex_female,0,1,1,1,0
sex_male,1,0,0,0,1
purch_0,1,1,1,1,1
purch_1,0,0,0,0,0
purch_2,0,0,0,0,0
purch_3,0,0,0,0,0


In [0]:
print(test_ds.shape)
print(X.shape)

(418, 25)
(891, 25)


## Model Building

What is stratified CV?
https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation 

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [0]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=5)

In [0]:
decision_tree_params ={'max_depth':list(range(1,5)), 'min_samples_leaf':list(range(1,5))}

In [0]:
from sklearn.model_selection import GridSearchCV
grid_search_tree=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid=decision_tree_params,n_jobs=-1,cv=skf,verbose=True)

In [0]:
grid_search_tree.fit(X,y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    1.5s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=5, shuffle=True),
       error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [1, 2, 3, 4], 'min_samples_leaf': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [0]:
print("Best Params: ",grid_search_tree.best_params_)

Best Params:  {'max_depth': 3, 'min_samples_leaf': 1}


In [0]:
print("Best Score: ",grid_search_tree.best_score_)

Best Score:  0.813692480359147


In [0]:
decision_tree_predictions=grid_search_tree.best_estimator_.predict(test_ds)

In [0]:
predicted_report=pd.DataFrame(decision_tree_predictions,columns=['survived'])

In [0]:
predicted_report.groupby('survived').agg({'survived':'count'})

,survived
survived,
0,307
1,111


##Conclusion

As per the above result the 111 person will servive out of 418 